In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
%load_ext memory_profiler

import matplotlib.pyplot as plt
import skimage.io
import numpy as np
import tifffile

import nima_io.read as ir

from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
simgsingle = str(imgsingle)
smcts = str(mcts)
sbigtiff = str(bigtiff)

## nima_io.read

| function   |  time | Pos in md | extra md | no errors | note                     |
|------------|-------|-----------|----------|-----------|--------------------------|
| read       |  4.68 | yes       | no       | no        |                          |
| read2      |  12.4 | no        | yes      | no        |                          |
| read_inf   |  4.46 | no        | no       | yes       | core md misses many keys |
| read_bf    | 0.487 | no        | no       | no        |                          |
| reaf_jb    |  4.47 | yes       | no       | no        |                          |
| read_wrap  |     - | yes       | no       | yes       | It calls read            |
|----
| read_pims  |  2.57 | yes       | no       | yes       | extra pims DIMS          |
| read_jpype | 0.263 | yes       | yes      | yes       |                          |

### read_jpype

In [6]:
%timeit ir.read_jpype(simg_void_tile)
%memit ir.read_jpype(simg_void_tile)

rjpype = ir.read_jpype(simg_void_tile)

233 ms ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 896.71 MiB, increment: 2.74 MiB


In [7]:
(core_md_jpype, (wr_jpype, type, extra_md_jpype)) = rjpype

In [11]:
read_dictionary = np.load("extra_md_read2.npy", allow_pickle=True).item()

sorted_dict = {k: read_dictionary[k] for k in sorted(read_dictionary.keys())}
sorted_dictJ = {k: extra_md_jpype[k] for k in sorted(extra_md_jpype.keys())}

In [13]:
sorted(extra_md_jpype.keys()) == sorted(read_dictionary.keys())

False

In [15]:
read_dictionary.keys() - extra_md_jpype.keys()

{'Creator', 'Format'}

In [17]:
read_dictionary["Format"], read_dictionary["Creator"]

('OME-TIFF', [((), 'FEI Munich GmbH, Live Acquisition, V2.5.0.6')])

In [18]:
read_dictionary.pop("Format")
read_dictionary.pop("Creator")

[((), 'FEI Munich GmbH, Live Acquisition, V2.5.0.6')]

In [19]:
sorted_dict = {k: read_dictionary[k] for k in sorted(read_dictionary.keys())}
for k in sorted_dict:
    if not sorted_dict[k] == sorted_dictJ[k]:
        print(k)

ChannelLightSourceSettingsAttenuation
PixelsBigEndian
PixelsInterleaved


In [21]:
print(
    read_dictionary["ChannelLightSourceSettingsAttenuation"],
    extra_md_jpype["ChannelLightSourceSettingsAttenuation"],
)
print(read_dictionary["PixelsBigEndian"], extra_md_jpype["PixelsBigEndian"])
print(read_dictionary["PixelsInterleaved"], extra_md_jpype["PixelsInterleaved"])

[((13, 2), 0.9)] [((13, 2), 0.8999999761581421)]
[((13,), False)] [((13,), 'False')]
[((13,), False)] [((13,), 'False')]


In [29]:
f = ir.stitch(core_md_jpype, wr_jpype, c=1, t=0)
skimage.io.imshow(f)

ValueError: One or more series doesn't have a single XYZ position.

In [45]:
core_md_jpype["series"][0]["PositionXYZ"]

{21.953, 50.62738, 81.83527}

### PIMS

Which is currently unable to download loci_tools.jar.

**I really like the frame metadata t_s, x_um, y_um and z_um.
Every array (2D, 3D, ..., nD) having those metadata in common are contained in the Frame obj: a numpy array with metadata(dict) and frame_no(int).**

Are fs.bundle_axes (fs.frame_shape), fs.iter_axes and fs.default_coords overcomplicated?

Anyway: iter=0 == iter=n which is at least unexpected.

In [ ]:
%timeit ir.read_pims(img_void_tile)
%memit ir.read_pims(img_void_tile)

rpims = ir.read_pims(img_void_tile)
rpims

### Update bioformats to scyjava

So to remove the annoying long warning, find the jar for loci and start stop JVM properly.

In [ ]:
bioformats.ImageReader(str(img_tile))

In [ ]:
import jpype, scyjava

scyjava.start_jvm()


def hello():
    JOptionPane = scyjava.jimport("javax.swing.JOptionPane")
    JOptionPane.showMessageDialog(None, "Hello world")


jpype.setupGuiEnvironment(hello)

In [2]:
# bioformats_jar
import jpype
import scyjava

scyjava.config.endpoints.append("ome:formats-gpl:6.7.0")
scyjava.start_jvm()
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")

In [3]:
# rdr = loci.formats.ChannelSeparator(loci.formats.ChannelFiller())
rdr = loci.formats.ImageReader()

In [12]:
rdr.setMetadataStore(loci.formats.MetadataTools.createOMEXMLMetadata())
rdr.setId(simg_void_tile)
xml_md = rdr.getMetadataStore()

In [17]:
# sr = image_reader.getSeriesCount()
md, mdd = ir.get_md_dict(xml_md, simg_void_tile)

In [21]:
# md['Format'] = rdr.format
# new core_md
# core_md = init_metadata(md["ImageCount"][0][1], rdr.format)
core_md = ir.init_metadata(md["ImageCount"][0][1], rdr.getFormat())

In [23]:
core_md

{'SizeS': 14, 'Format': 'OME-TIFF', 'series': []}

In [29]:
scyjava.jvm_started()

False

### TileSticher
   #+begin_src jupyter-python

     filepath = img_tile
     rdr = bioformats.formatreader.make_image_reader_class()()
     rdr.allowOpenToCheckType(True)

     clsOMEXMLService = javabridge.JClassWrapper('loci.formats.services.OMEXMLService')
     serviceFactory = javabridge.JClassWrapper('loci.common.services.ServiceFactory')()
     service = serviceFactory.getInstance(clsOMEXMLService.klass)
     metadata = service.createOMEXMLMetadata()
     rdr.setMetadataStore(metadata)
     rdr.setId(filepath)



     ts = javabridge.JClassWrapper('loci.formats.TileStitcher')(rdr)
     cs = javabridge.JClassWrapper('loci.formats.ChannelSeparator')(rdr)
     ##is_tile = ts.makeTileStitcher(rdr)
   #+end_src

   #+RESULTS:


   #+begin_src jupyter-python

     rdr.setId(filepath)

     s = 0
     root = metadata.getRoot()
     first_image = root.getImage(s)
     pixels = first_image.getPixels()

     # The plane data isn't in the planes, it's in the tiff data
     for idx in range(pixels.sizeOfTiffDataList()):
         tiffData = pixels.getTiffData(idx)
         c = tiffData.getFirstC().getValue().intValue()
         t = tiffData.getFirstT().getValue().intValue()
         print("TiffData: c=%d, t=%d" % (c, t))
   #+end_src

#+RESULTS:
#+begin_example
  TiffData: c=0, t=0
  TiffData: c=1, t=0
  TiffData: c=2, t=0
  TiffData: c=3, t=0
  TiffData: c=0, t=1
  TiffData: c=1, t=1
  TiffData: c=2, t=1
  TiffData: c=3, t=1
  TiffData: c=0, t=2
  TiffData: c=1, t=2
  TiffData: c=2, t=2
  TiffData: c=3, t=2
#+end_example



## ImageIO

Started comparing the available options to read microscopy file.

- mosaic

In [ ]:
from imageio.v3 import imread

%timeit imread(img_void_tile, index=13)
%memit imread(img_void_tile, index=13)
i = imread(img_void_tile, index=13)
i.shape

In [ ]:
i.nbytes, 512**2 * 3 * 4 * 2

It can read tif (tf8) files. Series might be passed using `index` (you need to know in advance).

## AICSImageIO

In [23]:
from aicsimageio import AICSImage

i = AICSImage(img_void_tile)
# i = AICSImage(img_void_tile, reconstruct_mosaic=True)
# i_lif = AICSImage(lif)

In [24]:
i.ome_metadata

OME(
   experiments=[{'description': '', 'experimenter_ref': {'id': 'Experimenter:0ec49b9d-d35f-46d9-9adc-9526e28d4f4d'}, 'type': [<Experiment_value.FOUR_DPLUS: 'FourDPlus'>], 'id': 'Experiment:344b0000-4d68-4181-9d41-1cf0eb64969b'}],
   plates=[<1 field_type>],
   experimenters=[{'id': 'Experimenter:0ec49b9d-d35f-46d9-9adc-9526e28d4f4d'}],
   instruments=[<1 field_type>],
   images=[<14 field_type>],
   structured_annotations=[<1 field_type>],
   creator='FEI Munich GmbH, Live Acquisition, V2.5.0.6',
)

In [26]:
i.scenes

('Image:99b33976-bed6-4899-82b1-ab02a8aa1cd0',
 'Image:be6d30af-41ed-4e5c-9c47-852f36b61456',
 'Image:327165fa-ccde-4824-9031-f01e1f69e9d3',
 'Image:4c24a448-6b56-4c94-9c93-76d0b7bafa61',
 'Image:0b97b10a-b6cb-4610-aea2-f37e14c228fb',
 'Image:b373be78-0fdd-47c0-8abf-3aedcd326dd1',
 'Image:2c74a9ce-16a4-4623-a111-3ece6c88c02e',
 'Image:714448cb-93cc-4ac1-9c30-68ca09798360',
 'Image:2f1dd659-32b6-4f62-83fd-22955e200d98',
 'Image:a671c95b-87ac-41ea-86ae-040c127d66bb',
 'Image:62a6a209-360c-4625-8562-89ff3fe8da14',
 'Image:8b48ee80-7dff-49fe-bea6-0054b702a24c',
 'Image:8257b63a-fba0-453e-a535-814138d2bc62',
 'Image:de6e3901-0832-4544-b41c-d8f0420195b2')

In [27]:
i.get_dask_stack()

dask.array<stack, shape=(14, 4, 3, 1, 512, 512), dtype=uint16, chunksize=(1, 1, 1, 1, 512, 512), chunktype=numpy.ndarray>

Mosaic stitch is not supported on tif files; so I will use my function relying on the PositionXYZ metadata.

## dask_image

In [ ]:
from dask_image.imread import imread

i = imread(img_void_tile)

In [ ]:
i

In [ ]:
imread(lif)

Somehow it uses bioformats and can handle lif. No mosaic, no metadata though.

**Pycroscopy** https://pypi.org/project/pycroscopy/ is not reading lif nor ome-tif at the moment.

**large-image[all]** failed to install.

**pyimagej** need conda?

## bioio-bioformats

In [ ]:
import bioio_ome_tiled_tiff

In [ ]:
bioio_ome_tiled_tiff.Reader(str(img_tile))

In [ ]:
import bioio_bioformats

im = bioio_bioformats.Reader(img_void_tile)
bioio_bioformats.ReaderMetadata(img_void_tile)

In [ ]:
im.xarray_dask_data

In [ ]:
i = bioio_bioformats.Reader(img_tile)
i.data.shape

In [ ]:
i.xarray_dask_data.attrs["processed"]

In [ ]:
import nima_io.read as ir

In [ ]:
ir.bioformats.get_omexml_metadata(img_tile)

In [ ]:
unp = i.xarray_dask_data.attrs["unprocessed"]
?ir.get_md_dict

In [ ]:
i.ome_metadata

In [ ]:
stk = i.get_dask_stack()

In [ ]:
stk.A

## bfio

In [ ]:
import bfio

bfio.BioReader(img_void_tile)

In [ ]:
rdr = bfio.BioReader(img_void_tile)
%timeit i = rdr.read()
i = rdr.read()
i.shape

In [ ]:
rdr.metadata

In [ ]:
rdr.ps_x

In [ ]:
rdr.close()

## PIMS

In [ ]:
import pims

%timeit fs = pims.Bioformats(img_void_tile)
fs = pims.Bioformats(img_void_tile)
fs.sizes

## PyOMETiff

In [ ]:
import pyometiff

%timeit rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)
rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)

In [ ]:
%timeit r = rdr.read()
res = rdr.read()

In [ ]:
res[2]

In [ ]:
pyometiff.OMETIFFReader._get_metadata_template()

## Final Note

I will keep 

0. Read
1. stitch
2. md_grouping

- impy
- napari.read
- pycromanager
- microscope
- python-microscopy